In [ ]:
%load_ext autoreload
%autoreload 2

In [62]:
import numpy as np
from model import build_model
from load_data import get_patches

original_patch_path_mapping = get_patches("data/Patches/PatchImage/")

PATCH_SHAPE = (224, 224, 3)

In [73]:
import pandas as pd

COLS = ["ID", "img_path", "N_category"]
clinical_array = pd.read_csv("data/train.csv")[COLS].values

In [66]:
model = build_model(PATCH_SHAPE)
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_10 (Conv2D)             (None, 221, 221, 36  1764        ['input[0][0]']                  
                                )                                                                 
                                                                                                  
 max_pooling2d_10 (MaxPooling2D  (None, 110, 110, 36  0          ['conv2d_10[0][0]']              
 )                              )                                                           

In [ ]:
# Call back setting

In [ ]:
# Generator
model_train_set, model_val_set = Get_train_valid_Path(train_set, train_percentage=0.9)
train_gen = DataGenerator(batch_size=1, shuffle=True).generate(model_train_set)

In [ ]:
model.fit_generator(
    generator=train_gen, 
    steps_per_epoch=len(model_train_set)//batch_size,
    epochs=args.max_epoch,
    validation_data=val_gen,
    validation_steps=len(model_val_set)//batch_size, 
    callbacks=callbacks
)